## Reviews from IMDb
E.g. the movie 'The Substance' has a url-page 'https://www.imdb.com/title/tt17526714/' where 'tt17526714' is the ImdbId.

In [2]:
# !pip install beautifulsoup4

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [30]:
def scrapeReviews(ImdbId, count=10, sort="submissionDate", ratingFilter=10, dir="asc") :
    
    # movie_url = f'https://www.imdb.com/title/{ImdbId}/reviews/_ajax?sort={sort}&dir={dir}&ratingFilter={ratingFilter}'
    movie_url = f'https://www.imdb.com/title/{ImdbId}/reviews/_ajax'
    # print(movie_url)
    r = requests.get(url=movie_url)
    if r.status_code != 200:
        raise Exception("Error in response. Bad URL?")
    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup.prettify())
    reviews = soup.find_all('div', {'class' : 'imdb-user-review'})
    
    data = []  # 'Move ID', 'Reviewer Name', 'Reviewer URL', 'Title', 'Review'
    num = 0
    for review in reviews:
        # print(BeautifulSoup(review.text, 'html.parser').prettify())
        # print(review)
        review_imdb = [ImdbId]
        review_imdb.append(review.find('span',{'class':'display-name-link'}).find('a').string.strip())
        review_imdb.append(review.find('span',{'class':'display-name-link'}).find('a')['href'].split('/')[2])
        review_imdb.append(review.find('a',{'class': 'title'}).string.strip())
        review_imdb.append(review.find('div',{'class' : 'show-more__control'}).get_text(strip=True))
        data.append(review_imdb)
        num += 1
        if num == count:
            break
    
    return data

data = scrapeReviews('tt17526714')  # Reviews for the substance
data

[['tt17526714',
  'Sugarfreee',
  'ur187764144',
  'A spiritual successor to Cronenberg style body horror',
  'Let me start out by saying: I love body horror and if you don\'t, or if you\'re squeamish, you might want to pass on this film. That said, I thought the balance between disturbing, impactful, intriguing, and disgusting was absolutely right on the money.In a world full where filler, botox, face lifts and implants, skin care routines, and obsession with youth are at their peak in media, THE SUBSTANCE calls into question whether the "new, better \'you\'" is really \'you\' - you are one, but perhaps not the same. It\'s clever, it\'s gross, it\'s pointed, and it\'s a fun watch. Moore and Qualley were an excellent duo and Quaid played a Hollywood sleezeball as believably as it could be done.'],
 ['tt17526714',
  'maria_nawrocka',
  'ur49877390',
  'Breasts just in the right place',
  '12 minutes of standing ovation during Cannes Film Festival premiere, Demi Moore with tears in her e

## Apply to top 250 movies

In [31]:
# Load the data
titles = pd.read_csv('data/imdb_top_250.csv')
titles.head()

,ID,Title
0,tt0111161,1. En verden udenfor
1,tt0068646,2. Godfather
2,tt0468569,3. The Dark Knight
3,tt0071562,4. Godfather del 2
4,tt0050083,5. Tolv vrede mænd


In [38]:
# review_table = []
# for id in tqdm(titles['ID']):
#     review_table += scrapeReviews(id)

# Create pandas dataframe
df = pd.DataFrame(review_table, columns=['Move ID', 'Reviewer Name', 'Reviewer URL', 'Title', 'Review'])
# Save the dataframe to a CSV file
df.to_csv('data/reviews_top_250.csv', index=False)
df.head()

,Move ID,Reviewer Name,Reviewer URL,Title,Review
0,tt0111161,hitchcockthelegend,ur16161013,Some birds aren't meant to be caged.,The Shawshank Redemption is written and direct...
1,tt0111161,Sleepin_Dragon,ur15311310,An incredible movie. One that lives with you.,It is no wonder that the film has such a high ...
2,tt0111161,EyeDunno,ur0265899,Don't Rent Shawshank.,I'm trying to save you money; this is the last...
3,tt0111161,kaspen12,ur1898687,A classic piece of unforgettable film-making.,"In its Oscar year, Shawshank Redemption (writt..."
4,tt0111161,alexkolokotronis,ur16117882,This is How Movies Should Be Made,This movie is not your ordinary Hollywood flic...


MINOR CHANGES TO COPY FROM https://github.com/pavan412kalyan/imdb-movie-scraper/blob/main/api/reviews.py

In [4]:
# -*- coding: utf-8 -*-

# this is for scraping reviews given imdb id - also with filters from api urls

import requests
from bs4 import BeautifulSoup
import re
import json
def scrapeReviews(ImdbId,sort="submissionDate",ratingFilter=10,dir="asc") :
    
    try :
#        movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews?"+"sort="+sort+"&dir="+dir+"&ratingFilter="+ratingFilter
         movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews/_ajax?"+"sort="+sort+"&dir="+dir+"&ratingFilter="+ratingFilter
         print(movie_url)
    except :
        
#        movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews"
         movie_url = "https://www.imdb.com/title/"+ImdbId+"/reviews"+"/_ajax"
        
    r = requests.get(headers={'User-Agent': 'Mozilla/5.0'},url=movie_url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'html.parser')
    try :
        reviews = soup.find_all('div',{'class' : 'imdb-user-review'})
    except :
        pass
        
    
    #review = soup.find('div',{'class' : 'imdb-user-review'})
    
    data = {}
    data['ImdbId'] = ImdbId
    reviews_text =[]
    for review in reviews :
        review_imdb={}
        
        ################
        try :
            review_imdb['reviewer_name']=review.find('span',{'class':'display-name-link'}).find('a').string.strip()
        except :
            review_imdb['reviewer_name']=""    
        ###############
        try :
            review_imdb['reviewer_url']=review.find('span',{'class':'display-name-link'}).find('a')['href']
        except:
            review_imdb['reviewer_url']=""
        ############
        try :
            review_imdb['data-review-id']=review['data-review-id']
        except :
            review_imdb['data-review-id']=""
            
        #############
        try:
            short_review =review.find('a',{'class': 'title'})
            text=short_review.string.strip()
            review_imdb['short_review']=text
        except :
            review_imdb['short_review']=""
    
        ######################
        try :
            full_review = review.find('div',{'class' : 'show-more__control'})
            text = full_review.string.strip()
            review_imdb['full_review']=text
        except :
             review_imdb['full_review']=""
        #############
        try :
            review_date = review.find('span',{'class' : 'review-date'})
            text=review_date.string.strip()
            review_imdb['review_date'] =text    
        except :
             review_imdb['review_date']  ="" 
        #######
        try :
            ratings_span = review.find('span',{'class' : 'rating-other-user-rating'})
            text=ratings_span.find('span').string.strip()
            review_imdb['rating_value']  = text      
        except :
            review_imdb['rating_value']  = "" 
        ##########
        reviews_text.append(review_imdb)    
    data['reviews']=reviews_text
    return data

scrapeReviews('tt17526714')

{'ImdbId': 'tt17526714',
 'reviews': [{'reviewer_name': 'Sugarfreee',
   'reviewer_url': '/user/ur187764144/?ref_=tt_urv',
   'data-review-id': 'rw10016456',
   'short_review': 'A spiritual successor to Cronenberg style body horror',
   'full_review': '',
   'review_date': '17 September 2024',
   'rating_value': '8'},
  {'reviewer_name': 'maria_nawrocka',
   'reviewer_url': '/user/ur49877390/?ref_=tt_urv',
   'data-review-id': 'rw9806680',
   'short_review': 'Breasts just in the right place',
   'full_review': '',
   'review_date': '24 May 2024',
   'rating_value': '8'},
  {'reviewer_name': 'julie-ford',
   'reviewer_url': '/user/ur18692252/?ref_=tt_urv',
   'data-review-id': 'rw10000136',
   'short_review': 'Future cult classic',
   'full_review': '',
   'review_date': '7 September 2024',
   'rating_value': '9'},
  {'reviewer_name': 'FeastMode',
   'reviewer_url': '/user/ur104603847/?ref_=tt_urv',
   'data-review-id': 'rw10020012',
   'short_review': 'My single most memorable movie ex

## Getting all movie IDs

In [5]:
# !pip install Scrapy

In [6]:
url = "https://www.imdb.com/title/"+'tt17526714'+"/reviews/_ajax?"
print(url)
r = requests.get(url=url)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

https://www.imdb.com/title/tt17526714/reviews/_ajax?
<div>
 <div class="lister-list">
  <div class="lister-item mode-detail imdb-user-review collapsable" data-review-id="rw10016456" data-vote-url="/title/tt17526714/review/rw10016456/vote/interesting">
   <div class="review-container">
    <div class="lister-item-content">
     <div class="ipl-ratings-bar">
      <span class="rating-other-user-rating">
       <svg class="ipl-icon ipl-star-icon" fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
        <path d="M0 0h24v24H0z" fill="none">
        </path>
        <path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z">
        </path>
        <path d="M0 0h24v24H0z" fill="none">
        </path>
       </svg>
       <span>
        8
       </span>
       <span class="point-scale">
        /10
       </span>
      </span>
     </div>
     <a class="title" href="/review/rw10016456/">
      A spiritual succ

## Data tables

In [7]:
import pandas as pd